In [ ]:
import numpy as np
import pandas as pd

%precision 4

In [200]:
userDframe = pd.read_csv('data.csv')
cleanedData = cleanAndNormalizeData(userDframe)

#userList=list(set(cleanedData['user_id']))
#dealList=list(set(cleanedData['deal_id']))

#print(len(userList))

cleanedData.head()
cleanedData.shape
iniData = randomRating(cleanedData)
iniData.to_csv('out.csv', encoding='utf-8', index=False)

In [152]:
#Cleaning data
#Dropping rows where impression_time<=0; because without seeing the user can't take any swipe action
#Dropping rows where impression_time >= 90000ms; this will be done because almost 99% values are in this range(<100000ms).

def cleanAndNormalizeData(userDataframe):
    indexNames = userDataframe[(userDataframe['impression_time'] <= 0) | (userDataframe['impression_time'] > 100000)].index
    modifiedData= userDataframe.drop(indexNames, axis=0)
    modifiedData['swipe_direction'].replace('SWIPED_OUT',-1,inplace=True)
    modifiedData['swipe_direction'].replace('SWIPED_IN',1,inplace=True)
    
    modifiedNormalizedData=getZScore(modifiedData)
    return modifiedNormalizedData

In [128]:
#Using Z-score normaliztion to normalize the impression_time column
def getZScore(modifiedData):
    z_score = (modifiedData['impression_time'] - modifiedData['impression_time'].mean())/modifiedData['impression_time'].std()
    modifiedData['z_score']=z_score
    return modifiedData

In [196]:
def randomRating(cleanedData):
    cleanedData['rating']=np.random.randint(4000,5000,cleanedData.shape[0])
    #if(cleanedData.loc[(cleanedData.swipe_direction == 1)]:
        #cleanedData['rating'] = cleanedData(np.random.uniform(low=0.45, high=0.5))
    #elif(cleanedData['swipe_direction'] == -1):
        #cleanedData['rating'] = cleanedData(np.random.uniform(low=0.45, high=0.5))
    cleanedData['rating']=cleanedData['rating']/10000   
        
        
    return cleanedData